In [1]:
#;.pykx.disableJupyter()

In [2]:
# https://code.kx.com/pykx/3.0/examples/jupyter-integration.html#q-first-mode
import pykx as kx
kx.util.jupyter_qfirst_enable()

PyKX now running in 'jupyter_qfirst' mode. All cells by default will be run as q code. 
Include '%%py' at the beginning of each cell to run as python code. 


In [3]:
system"cd ",.trn.nbdir:$["/"=first v;"";getenv[`HOME],"/"],v:first system "dirname '",getenv[`JPY_SESSION_NAME],"'"
\l scripts/loaddata.q

"Initializing variables"
"Loaded Weather CSV"
"Loaded Taxi Trips partitioned DB"
"Defining exercise results"
"Ready"


**Learning objectives**

To understand:

* What are lists?
* Casting
* Obtaining random data
* List amendment
* Dictionaries
* Tables

# Lists

[code.kx - q4m - lists](https://code.kx.com/q4m3/3_Lists/)

So far we have seen how a _table_ is a natural fit for storing and analyzing huge amounts of data. Under the covers though, q exploits a more fundamental data structure to build the table: the _list_. A large part of the performance edge of kdb+/q comes from its ability to work directly with lists – every user should be familiar with them!

To get our hands on our first list, consider the `trips` table that we already met. If all you care about is the amount people paid for their journey, we can inspect the `fare` column. You have already seen we can `select` out just the column of interest, with an optional Where phrase.

In [4]:
vtsfares:select fare from trips where date = 2009.01.01, vendor=`VTS
vtsfares

fare
----
5.7 
4.9 
4.9 
4.5 
4.9 
15.3
3.7 
8.5 
13.3
11.3
22.5
6.9 
16.5
24.1
9.3 
9.3 
6.1 
8.1 
5.7 
8.5 
..


Since tables in kdb+/q are *column-oriented*, columns can be extracted simply by indexing into the table with the column name, returning the column as a contiguous vector, or *list*. 

In [5]:
fares: vtsfares`fare
fares

5.7 4.9 4.9 4.5 4.9 15.3 3.7 8.5 13.3 11.3 22.5 6.9 16.5 24.1 9.3 9.3 6.1 8.1..


To check that we've got what we expect, we can use the `type` operator:

In [6]:
type fares

9h


The number is positive and under 20: we have a *simple list*. 
In a simple list, the items all have the same type.

When a list has items of different types, it is referred to as a *general list*. 
For example, a pair representing the taxi company and the fare paid could look like:

In [7]:
general:(`VTS;23.45);
general

`VTS
23.45


While simple lists always have strictly positive values returned by type, general lists always have type zero.

In [8]:
type general

0h


Joining entities of different types with the comma operator will produce a mixed list

In [9]:
general:2018.01.01,102,`hello,enlist "world"
general
type general

2018.01.01
102
`hello
"world"
0h


A list can be _empty_ - if there had been a typo in the select statement, and we inquired about a non-existent cab company, we would see:

In [10]:
svtfares:select fare from trips where month=2009.01m, vendor=`SVT
svtfares`fare

`float$()


## Casting

From the result above you can see `$` used to [cast](https://code.kx.com/q/ref/cast/) an empty list. 

When working with data, it is often necessary to cast (change) the data from one type e.g. a time like `09:30:00` to another e.g. a datetime like `2020.05.19T09:30:00`. We can use the `$` to cast a non-textual data type to another data type: 

In [11]:
`float$1 2 //using it's symbol name 
"f"$1 2  //using it's character letter
9h$1 2   //using it's short value

1 2f
1 2f
1 2f


We can create an empty list as a general (i.e.untyped) list, or create a typed empty list:

In [12]:
() //general list 
`long$() //list of type long

()
`long$()


We can look at the results just by passing the variable name `fares`:

In [13]:
fares

5.7 4.9 4.9 4.5 4.9 15.3 3.7 8.5 13.3 11.3 22.5 6.9 16.5 24.1 9.3 9.3 6.1 8.1..


Yikes! That's a lot of screenspace to waste. 
To save the electrons, we can just look at the first few elements using the `sublist` operator:

In [14]:
10 sublist fares

5.7 4.9 4.9 4.5 4.9 15.3 3.7 8.5 13.3 11.3


sublist is clever - to get the _last_ few elements, all you have to do is give a negative number:

In [15]:
-10 sublist fares

12.9 4.9 44.5 5.3 19.3 9.3 8.9 45 9.7 4.5


##### Exercise 13
- Use sublist to get the second 10 elements in the list

In [16]:
-10 sublist 20 sublist fares
//alternative way - 10 10 sublist fares

22.5 6.9 16.5 24.1 9.3 9.3 6.1 8.1 5.7 8.5


In [17]:
// Enter your code here 

In [18]:
ex13[] //check correct output

22.5 6.9 16.5 24.1 9.3 9.3 6.1 8.1 5.7 8.5


Sublist has a nice property that the number of elements it returns is _capped_ at the size of the list that it operates on. In comparison the [Take operator `#`](https://code.kx.com/q/ref/take/) returns exactly the number of items you specify:

In [19]:
count 10000000 # fares
count 10000000 sublist fares

10000000
124208


Let’s put a bit of structure on our list: make a _sorted_ copy of it that we can play with. 
The [`asc` keyword](https://code.kx.com/q/ref/asc/) does this:

In [20]:
sortedFares:asc fares

`sortedFares` has the same `count` and `type` as fares, but now is _sorted_ in ascending order. If you looked only at the first elements of this list, you might conclude that cab journeys in NYC are great value!

In [21]:
10 sublist sortedFares

2.5 2.5 2.5 2.5 2.5 2.5 2.5 2.5 2.5 2.5


##### Exercise 14
- Use sublist to obtain the 10 highest values from the sorted List

In [22]:
-10 sublist sortedFares

104 104 108.5 112.5 120 122 128 134.3 190 200


In [23]:
// Enter your code here 

In [24]:
ex14[] //check correct output

104 104 108.5 112.5 120 122 128 134.3 190 200


## Obtaining random data

An easy trap to fall into – we extracted an _unrepresentative sample_. 
To pick ten _random_ records from the list, we can use the [Roll `?` operator](https://code.kx.com/q/ref/deal/)

In [25]:
sampleFares:10?sortedFares;
sampleFares

8.5 11.7 8.5 7.3 5.3 45 7.7 4.9 3.7 18.9


Lists support _random access_. To pick out the 10th element of a list, we use:

In [26]:
fares[9]

11.3


The preceding yields an atom, as can be verified by using `type`. A similar approach works for lists of indexes:

In [27]:
fares[0 1 2 3 4 5 6 7 8 9]

5.7 4.9 4.9 4.5 4.9 15.3 3.7 8.5 13.3 11.3


<img src="images/qbies.png" width="50px" style="width: 50px;padding-right:5px;padding-top:20px;padding-left:5px;" align="left"/><p style='color:#273a6e'><i> Normally, you would use the [`til` keyword](https://code.kx.com/q/ref/til/), to get the list of the first N ints, starting at zero. (As you have seen, q uses zero indexing.) </i></p>

In [28]:
til 10
fares[til 10]

0 1 2 3 4 5 6 7 8 9
5.7 4.9 4.9 4.5 4.9 15.3 3.7 8.5 13.3 11.3


##### Exercise 15
- Extract the 11th to the 20th elements from the fares list using the til keyword

In [29]:
fares[10 + til 10]

22.5 6.9 16.5 24.1 9.3 9.3 6.1 8.1 5.7 8.5


In [30]:
// Enter your code here

In [31]:
ex15[] //check correct output

22.5 6.9 16.5 24.1 9.3 9.3 6.1 8.1 5.7 8.5


##### Exercise 16
- Use indexing to find the middle value in the `sortedFares` list. 

In [32]:
sortedFares [`long$(count sortedFares)%2]

6.9


In [33]:
// Enter your code here

In [34]:
ex16[] //check correct output

6.9


In the case of a simple list, if the index used is too high, a _null_ of the list’s type is returned.

In [35]:
sortedFares[count sortedFares]
sortedFares[-1+count sortedFares]  // index from 0 to N-1

0n
200f


The below code block will also obtain the first value from the list: 

In [36]:
1 sublist sortedFares
first sortedFares

,2.5
2.5


<img src="images/qbies.png" width="50px" style="width: 50px;padding-right:5px;padding-top:20px;padding-left:5px;" align="left"/><p style='color:#273a6e'><i> Notice the difference between what is returned by `1 sublist sortedFares` and `first sortedFares`. The former returns a one-item list and the second an atom. You can see below how q displays them on the console </i></p>

[`enlist`](https://code.kx.com/q/ref/enlist/) returns a list containing the argument passed to it

Join `()` to an atom to make a one-item list

In [37]:
enlist 499
(),499

,499
,499


## List Amendment
A simple list can be indexed into using the `@` operator:

In [38]:
2* til 5
@[sampleFares;(2*til 5)]

0 2 4 6 8
8.5 8.5 5.3 7.7 3.7


The `@` operator can be applied with further arguments so that the list can be altered. Below we replace the items at positions `2*til 5` with `99f`.

In [39]:
// index into sampleFares
// using list of indexes (2*til 5)
// assign these values - :
// the value 99f
@[sampleFares;(2*til 5);:;99f]  

99 11.7 99 7.3 99 45 99 4.9 99 18.9


Below we use `+` instead of `:` – instead of replacing the items, we add `99f` to them.

In [40]:
@[sampleFares;(2*til 5);+;99f]

107.5 11.7 107.5 7.3 104.3 45 106.7 4.9 102.7 18.9


The above is not a persistent change - it will make a copy of the `fares` list with a single value changed and display the result at the terminal, but there is no change to the `fares` list. 

In [41]:
sampleFares  // original list not updated

8.5 11.7 8.5 7.3 5.3 45 7.7 4.9 3.7 18.9


To persist the change, prefix the name of the list with a back-tick; or assign the result to a name:

In [42]:
test:@[fares;(2*til 4);:;0Nf]
test
@[`fares;(2*til 4);:;0Nf]
fares

0n 4.9 0n 4.5 0n 15.3 0n 8.5 13.3 11.3 22.5 6.9 16.5 24.1 9.3 9.3 6.1 8.1 5.7..
fares
0n 4.9 0n 4.5 0n 15.3 0n 8.5 13.3 11.3 22.5 6.9 16.5 24.1 9.3 9.3 6.1 8.1 5.7..


Extend a list by appending to to it using the [Join operator `,`](https://code.kx.com/q/ref/join/).

In [43]:
fares,:12.34
-10#fares    // inspect the end of the list to see the appended value

4.9 44.5 5.3 19.3 9.3 8.9 45 9.7 4.5 12.34


Perhaps some data has been lost lost, or otherwise corrupted. kdb+/q handles null values. Is this a problem for us?

In [44]:
any null fares

1b


This is exactly equivalent to using `any[null[fares]]` – but perhaps a little cleaner? Your mileage may vary!

The [`null` keyword](https://code.kx.com/q/ref/null/) flags nulls.

In [45]:
where null fares

0 2 4 6


##### Exercise 17

Amend the fares list to replace the null values to be equal to the average value.

In [46]:
@[fares;where null fares;:;avg fares]

9.434303 4.9 9.434303 4.5 9.434303 15.3 9.434303 8.5 13.3 11.3 22.5 6.9 16.5 ..


In [47]:
// Enter your code here

In [48]:
ex17[] //check correct output

9.434303 4.9 9.434303 4.5 9.434303 15.3 9.434303 8.5 13.3 11.3 22.5 6.9 16.5 ..


# Dictionaries 
[Dictionaries](https://code.kx.com/q/basics/dictsandtables/) are first-class objects in q. (They are known as *hashmaps* in some other languages.) 

Use the [Dict operator `!`](https://code.kx.com/q/ref/dict/) to make a dictionary from a list of keys and a list of values.

In [49]:
d:`a`b!0 1
d

a| 0
b| 1


We can access and update existing values being passing in the key to the variable name:

In [50]:
d[`a]
d[`a]:2
d

0
a| 2
b| 1


We can also add keys to the existing dictionary:

In [51]:
d[`c]:3 // add a new key/value pair to d
d

a| 2
b| 1
c| 3


A dictionary can be joined to another dictionary. Below we have two examples:
1. Add values of two dictionaries
2. Join two dictionaries, prioritising values from the right-hand dictionary

In [52]:
d1:`a`b`c`d!5 6 7 8

In [53]:
d+d1 // add values for common keys
d,d1 // catenation - updates values for common keys, inserts new keys. Typical application is updating a snapshot with deltas.

a| 7
b| 7
c| 10
d| 8
a| 5
b| 6
c| 7
d| 8


# Tables 
Tables are first-class objects in q. Any list of 'like dictionaries' (meaning mulitple dictionaries with the same key) is a table. They can also be constructed with table notation or from column dictionaries.

1. Creating a table from a list of like dictionaries

In [54]:
(`a`b!0 1;`a`b!2 3)

a b
---
0 1
2 3


2. Creating a table with [table notation](https://code.kx.com/q/kb/faq/#table-notation)

In [55]:
([]a:0 2;b:1 3)

a b
---
0 1
2 3


3. Creating a table from a [column dictionary](https://code.kx.com/q/kb/faq/#flip-a-column-dictionary). A table is a transpose (flip) of a conforming dictionary (key of symbols, value of list of equal length lists).

In [56]:
flip `a`b!(0 2;1 3) 

a b
---
0 1
2 3


We can also add tables together

In [57]:
([]a:0 2;b:1 3)+([]a:4 5;b:6 7)

a b 
----
4 7 
7 10


Tables can be keyed. Here are two of the ways to create a [keyed table](https://code.kx.com/q/kb/faq/#keyed-tables).

1. Specify key columns with the [`xkey` keyword](https://code.kx.com/q/ref/xkey/)

In [58]:
k:`a xkey ([]a:0 2;b:1 3)
k

a| b
-| -
0| 1
2| 3


2. Specify key columns in the table notation.

In [59]:
([a:0 2]b:1 3)
([a:0 2;b:1 3]c:4 5)

a| b
-| -
0| 1
2| 3
a b| c
---| -
0 1| 4
2 3| 5


Working with a keyed table is similar to working with a dictionary. We obtain the keys and values with `key` and `value`:

In [60]:
key k
value k

a
-
0
2
b
-
1
3


A keyed table is a dictionary where both key and values are tables:

In [61]:
key[k]!value k

a| b
-| -
0| 1
2| 3


And as such, we can perform lookups on the keys to obtain values based on the keys:

In [62]:
k([]a:0 1 2)
([]a:0 1 2)#k

b
-
1
 
3
a| b
-| -
0| 1
1|  
2| 3


##### Exercise 18

a.  Create a dictionary with keys, `a`, `b`, and `c`, and assign to each key a list of three random ints.

In [63]:
dict:`a`b`c!(3?10i;3?10i;3?10i)
dict

a| 8 4 2
b| 2 9 3
c| 0 2 3


In [64]:
// Enter your code here 

In [65]:
exer18_a[] //check correct output

a| 6 2 3
b| 4 0 8
c| 1 3 2


Add a new key, `d` with double the values of key `a`.

In [66]:
dict[`d]:2*dict[`a]
dict

a| 6  2 3
b| 4  0 8
c| 1  3 2
d| 12 4 6


In [67]:
// Enter your code here

In [68]:
exer18_b[] //check correct output

a| 6  2 3
b| 4  0 8
c| 1  3 2
d| 12 4 6


c. Make a table from the dictionary

In [69]:
tab:flip dict
tab

a b c d 
--------
6 4 1 12
2 0 3 4 
3 8 2 6 


In [70]:
// Enter your code here 

In [71]:
exer18_c[] //check correct output

a b c d 
--------
6 4 1 12
2 0 3 4 
3 8 2 6 


d. Make a new table by joining the table to itself

In [72]:
tab2:tab,tab
tab2

a b c d 
--------
6 4 1 12
2 0 3 4 
3 8 2 6 
6 4 1 12
2 0 3 4 
3 8 2 6 


In [73]:
// Enter your code here

In [74]:
exer18_d[] //check correct output

a b c d 
--------
6 4 1 12
2 0 3 4 
3 8 2 6 
6 4 1 12
2 0 3 4 
3 8 2 6 


e. Make column `b` the key of this new table

In [75]:
tabKeyed:`b xkey tab2
tabKeyed

b| a c d 
-| ------
4| 6 1 12
0| 2 3 4 
8| 3 2 6 
4| 6 1 12
0| 2 3 4 
8| 3 2 6 


In [76]:
// Enter your code here 

In [77]:
exer18_e[] //check correct output

b| a c d 
-| ------
4| 6 1 12
0| 2 3 4 
8| 3 2 6 
4| 6 1 12
0| 2 3 4 
8| 3 2 6 


f. Compare the types of all the generated tables and dictionaries. What do you notice?

In [78]:
type each (dict;tab;tabKeyed)

99 98 99h


In [79]:
// Enter your code here 

In [80]:
// Run this cell to compare results
exer18_f[]

dict| 99
tab | 98
tab2| 98
tabk| 99
